<a href="https://colab.research.google.com/github/Rohan2139/MALWARE_ANALYSIS1/blob/main/Malware_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pefile

def analyze_pe_file(file_path):
    try:
        pe = pefile.PE(file_path)

        print("File Info:")
        print("-----------")
        print("Machine type: ", hex(pe.FILE_HEADER.Machine))
        print("Number of Sections: ", pe.FILE_HEADER.NumberOfSections)
        print("Compile Time: ", pe.FILE_HEADER.TimeDateStamp)
        print("Entry Point: ", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))
        print("\nSections:")
        print("---------")
        for section in pe.sections:
            print(section.Name.decode().rstrip('\x00'), "\tVirtual Size:", section.Misc_VirtualSize, "\tRaw Size:", section.SizeOfRawData)

        print("\nImported Functions:")
        print("-------------------")
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            print(entry.dll.decode())
            for imp in entry.imports:
                print("\t", imp.name.decode() if imp.name else "Ordinal:", hex(imp.address))

        print("\nExported Functions:")
        print("-------------------")
        if hasattr(pe, 'DIRECTORY_ENTRY_EXPORT'):
            for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
                print("\t", exp.name.decode())

    except pefile.PEFormatError as e:
        print(f"Error: {e}")

if _name_ == "_main_":
    file_path = input("Enter the path of the file to analyze: ")
    analyze_pe_file(file_path)

In [ ]:
import pefile
import matplotlib.pyplot as plt

def analyze_pe_file(file_path):
    try:
        pe = pefile.PE(file_path)

        # Plotting section sizes
        section_names = []
        virtual_sizes = []
        raw_sizes = []
        for section in pe.sections:
            section_names.append(section.Name.decode().rstrip('\x00'))
            virtual_sizes.append(section.Misc_VirtualSize)
            raw_sizes.append(section.SizeOfRawData)

        plt.figure(figsize=(10, 6))
        plt.bar(section_names, virtual_sizes, color='b', alpha=0.5, label='Virtual Size')
        plt.bar(section_names, raw_sizes, color='r', alpha=0.5, label='Raw Size')
        plt.xlabel('Sections')
        plt.ylabel('Size')
        plt.title('Section Sizes')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        # Plotting imported functions
        imported_dlls = []
        imported_functions = []
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            imported_dlls.append(entry.dll.decode())
            imported_functions.append(len(entry.imports))

        plt.figure(figsize=(10, 6))
        plt.bar(imported_dlls, imported_functions, color='g', alpha=0.7)
        plt.xlabel('Imported DLLs')
        plt.ylabel('Number of Functions')
        plt.title('Imported Functions by DLL')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.show()

        # Exported functions
        exported_functions = []
        if hasattr(pe, 'DIRECTORY_ENTRY_EXPORT'):
            for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
                exported_functions.append(exp.name.decode())

        if exported_functions:
            print("\nExported Functions:")
            print("-------------------")
            for exp_func in exported_functions:
                print("\t", exp_func)

    except pefile.PEFormatError as e:
        print(f"Error: {e}")

if _name_ == "_main_":
    file_path = input("Enter the path of the file to analyze: ")
    analyze_pe_file(file_path)

In [ ]:
import pefile
import os
import hashlib
import magic
import sys

# Function to calculate MD5 hash of a file
def calculate_md5(file_path):
    with open(file_path, 'rb') as f:
        md5 = hashlib.md5()
        while chunk := f.read(4096):
            md5.update(chunk)
    return md5.hexdigest()

# Function to get file type
def get_file_type(file_path):
    mime = magic.Magic(mime=True)
    return mime.from_file(file_path)

# Function to analyze PE file
def analyze_pe_file(file_path):
    try:
        pe = pefile.PE(file_path)

        # Basic file info
        print("File Info:")
        print("-----------")
        print("File Name:", os.path.basename(file_path))
        print("MD5 Hash:", calculate_md5(file_path))
        print("File Type:", get_file_type(file_path))
        print("Machine Type:", hex(pe.FILE_HEADER.Machine))
        print("Number of Sections:", pe.FILE_HEADER.NumberOfSections)
        print("Compile Time:", pe.FILE_HEADER.TimeDateStamp)
        print("Entry Point:", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))

        # Sections
        print("\nSections:")
        print("---------")
        for section in pe.sections:
            print("Name:", section.Name.decode().rstrip('\x00'))
            print("\tVirtual Size:", section.Misc_VirtualSize)
            print("\tRaw Size:", section.SizeOfRawData)
            print("\tEntropy:", section.get_entropy())

        # Imported functions
        print("\nImported Functions:")
        print("-------------------")
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            print(entry.dll.decode())
            for imp in entry.imports:
                print("\t", imp.name.decode() if imp.name else "Ordinal:", hex(imp.address))

        # Exported functions
        print("\nExported Functions:")
        print("-------------------")
        if hasattr(pe, 'DIRECTORY_ENTRY_EXPORT'):
            for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
                print("\t", exp.name.decode())

    except pefile.PEFormatError as e:
        print(f"Error: {e}")

if _name_ == "_main_":
    if len(sys.argv) != 2:
        print("Usage: python malware_analysis.py <file_path>")
        sys.exit(1)

    file_path = sys.argv[1]
    if not os.path.exists(file_path):
        print("File not found!")
        sys.exit(1)

    analyze_pe_file(file_path)